In [2]:
import torch
from transformers import BertModel


In [ ]:
data_folder=os.getcwd()
test_set="book30-listing-test.csv"
train_set="book30-listing-train.csv"

test=pd.read_csv(os.path.join(data_folder+"/"+test_set), sep=",", encoding='latin_1') #'unicode_escape'
train=pd.read_csv(os.path.join(data_folder+"/"+train_set), sep=",", encoding='latin_1')
#setting up columns' names
test=test.set_axis(["AMAZON INDEX (ASIN)","FILENAME","IMAGE URL","TITLE","AUTHOR","CATEGORY ID","CATEGORY"], axis=1, inplace=False)
train=train.set_axis(["AMAZON INDEX (ASIN)","FILENAME","IMAGE URL","TITLE","AUTHOR","CATEGORY ID","CATEGORY"], axis=1, inplace=False)

#selecting title and category id
test_set = test[['TITLE','CATEGORY ID']]
train_set = train[['TITLE','CATEGORY ID']]
#train_set2 = train_set.sample(frac=1).reset_index(drop=True) #shuffling for val data set
#print(train_set)
#print(train_set2.head())


In [ ]:
#checking for the longest title -> we want to set padding to max_length instead of 512
length_of_the_messages_test = test_set['TITLE'].str.split("\\s+")
length_of_the_messages_train = train_set['TITLE'].str.split("\\s+")

print(f"Max number of words in test_set: {length_of_the_messages_test.str.len().max()}")
print(f"Max number of words in train_set: {length_of_the_messages_train.str.len().max()}")
print("Index = ", length_of_the_messages_train.str.len().idxmax())

print(length_of_the_messages_train[4984])

class Dataset(torch.utils.data.Dataset):

    def __init__(self, X):

        self.labels = [label for label in X['CATEGORY ID']]  #list
        self.texts = [tokenizer(title, truncation=True, padding='max_length', max_length = 74,  #512
                    return_tensors="pt") for title in X['TITLE']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased',
                                         do_lower_case=True)
length = len(train_set) + len(test_set)
##print(len(test_set))
#print(length)
print(0.8*length, 0.1* length, 0.1*length)

train_s=train_set[:45598] #change to train_set2
val_s=train_set[45598:] #change to train_set2

print(f'LENGTH OF VALIDATION SET: {len(val_s)}, \
      LENGTH OF TEST SET: {len(test_set)},  \
      LENGTH OF TRAIN SET: {len(train_s)}. ')

train, val = Dataset(train_s), Dataset(val_s)

In [3]:
model_text = BertModel.from_pretrained('bert-base-cased', num_labels=30, output_attentions = False, 
   output_hidden_states = False)
model_text.load_state_dict(torch.load('best_model.pth'),strict=False)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.

_IncompatibleKeys(missing_keys=['embeddings.position_ids', 'embeddings.word_embeddings.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.LayerNorm.weight', 'embeddings.LayerNorm.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.1.attention.self.query.we

In [ ]:
data_dir = './images'

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.pretrained = torchvision.models.resnet50(pretrained=False)
        self.pretrained.fc = torch.nn.Identity()
        
        self.linear = torch.nn.Linear(2048, 30)
       


    def forward(self, x):        
        x = self.pretrained(x)
        x = self.linear(x)
        return x


In [ ]:
#model_img = CNN()
#model_img = model_img.cuda()
#model_img.load_state_dict(torch.load('CNN_model.pth'))
